In [1]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

device = torch.device('cpu')

In [2]:
num_epochs = 15
batch_size = 128
learning_rate = 0.001

In [3]:
train_dataset = torchvision.datasets.MNIST(root='../../data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='../../data/', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle = False)

100.0%


Extracting ../../data/MNIST\raw\train-images-idx3-ubyte.gz to ../../data/MNIST\raw



100.0%

Extracting ../../data/MNIST\raw\train-labels-idx1-ubyte.gz to ../../data/MNIST\raw




4.0%

100.0%


Extracting ../../data/MNIST\raw\t10k-images-idx3-ubyte.gz to ../../data/MNIST\raw



100.0%

Extracting ../../data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data/MNIST\raw



In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(1,32,3,1)
        self.conv2 = nn.Conv2d(32,64,3,1)
        self.dense1 = nn.Linear(12*12*64, 128)
        self.dense2 = nn.Linear(128,10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = F.dropout(x,0.25)
        x = x.view(-1, 12*12*64)
        x = F.relu(self.dense1(x))
        x = F.dropout(x,0.25)
        x = self.dense2(x)
        return F.log_softmax(x,dim=1)
        

In [11]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

    #Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
    
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4%}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/15], Step [100/469], Loss: 25.2523%
Epoch [1/15], Step [200/469], Loss: 8.5622%
Epoch [1/15], Step [300/469], Loss: 12.1812%
Epoch [1/15], Step [400/469], Loss: 9.7914%
Epoch [2/15], Step [100/469], Loss: 3.9408%
Epoch [2/15], Step [200/469], Loss: 11.1880%
Epoch [2/15], Step [300/469], Loss: 6.9643%
Epoch [2/15], Step [400/469], Loss: 5.6727%
Epoch [3/15], Step [100/469], Loss: 1.9566%
Epoch [3/15], Step [200/469], Loss: 2.7167%
Epoch [3/15], Step [300/469], Loss: 5.1164%
Epoch [3/15], Step [400/469], Loss: 4.7816%
Epoch [4/15], Step [100/469], Loss: 8.4234%
Epoch [4/15], Step [200/469], Loss: 2.6111%
Epoch [4/15], Step [300/469], Loss: 3.9878%
Epoch [4/15], Step [400/469], Loss: 11.1947%
Epoch [5/15], Step [100/469], Loss: 2.3188%
Epoch [5/15], Step [200/469], Loss: 4.4662%
Epoch [5/15], Step [300/469], Loss: 6.0193%
Epoch [5/15], Step [400/469], Loss: 5.1142%
Epoch [6/15], Step [100/469], Loss: 2.1056%
Epoch [6/15], Step [200/469], Loss: 3.1712%
Epoch [6/15], Step [300/469]

In [14]:
from sklearn.metrics import roc_auc_score
preds = []
y_true = []
#test the model 
model.eval() #set model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        detached_pred = predicted.detach().cpu().numpy()
        detached_label = labels.detach().cpu().numpy()
        for f in range(0,len(detached_pred)):
            preds.append(detached_pred[f])
            y_true.append(detached_label[f])
    print('Test Accuracy of the model on the 10000 test images: {:.2%}' .format(correct / total))
    preds = np.eye(10)[preds]
    y_true = np.eye(10)[y_true]
    auc = roc_auc_score(preds, y_true)
    print("AUC: {:.2%}".format(auc))

Test Accuracy of the model on the 10000 test images: 98.72%
AUC: 99.29%
